## **Simulation Set-Up**

In [1]:
import os

import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    AddMultibodyPlantSceneGraph,
    DiagramBuilder,
    LoadModelDirectives,
    LoadModelDirectivesFromString,
    Parser,
    ProcessModelDirectives,
    RigidTransform,
    RollPitchYaw,
    Simulator,
    StartMeshcat,
)
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.systems.analysis import Simulator
from pydrake.visualization import ModelVisualizer

from manipulation import running_as_notebook
from manipulation.station import LoadScenario, MakeHardwareStation
from manipulation.utils import ConfigureParser

In [2]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


# Simulation Set Up 

Generating a useful simulation scene is the first step in successful manipulation projects. In this exercise, we will learn how to add the necessary objects to set-up a scene.

**These are the learning goals of this exercise:**
1. Be able to work with objects written in robot description formats such as URDFs and SDFs.
2. Understand how to view and edit collision geometries
3. Understand how to add objects to Drake to create a useful manipulation scene.

A great resource is [Authoring a Multibody Simulation tutorial](https://deepnote.com/workspace/Drake-0b3b2c53-a7ad-441b-80f8-bf8350752305/project/Tutorials-2b4fc509-aef2-417d-a40d-6071dfed9199/notebook/authoring_multibody_simulation-add293478aac40a984845aa3705eefdd?). Parts of this notebook were taken from it.

## Introducing Relevant File Formats

The most important formats for creating multibody scenarios in Drake are the [Unified Robot Description Format (URDF)](http://wiki.ros.org/urdf) and the [Simulation Description Format (SDFormat)](http://sdformat.org/).

URDF and SDF are both XML formats that can specify the kinematic and dynamic properties of robots or objects. URDF can only specify the kinematic and dynamic properties of a single robot in isolation. It can't specify joint loops or things that are not robots, such as lights, heightmaps, etc. 

SDFormat is a complete description for everything from the world level down to the robot level. Being more scalable, it is the recommended format. However, they should be mostly interchangeable with small syntax differences.

In a high-level sense, you express different components of your robot using `<link>` tags and connect them via `<joint>` tags. Each `<link>` has three major subtags, `<visual>`, `<collision>`, and `<inertial>`, for its visualization, planning/collision checking, and dynamics aspects. For `<visual>` and `<collision>`, you can either use primitive shapes (box, sphere, cylinder, etc.) or tailored meshes (.obj, .gltf) to represent the underlying geometry. 

Here are two useful resources for [URDF](https://wiki.ros.org/urdf/Tutorials/Building%20a%20Visual%20Robot%20Model%20with%20URDF%20from%20Scratch) and [SDFormat](https://classic.gazebosim.org/tutorials?tut=build_model) creation. ([Another SDF reference](http://sdformat.org/spec))

Meshes and primitive shapes describe the geometry of an object. The supported types are described [here](https://drake.mit.edu/doxygen_cxx/group__geometry__file__formats.html). For meshes, glTF (.gltf) is the preferred format in Drake for visualization. Wavefront OBJ (.obj) is another well-supported alternative. obj files will often come with material descriptions or (.mtl) files that they reference. If you have other file formats (eg .stl, .dae), [Meshlab](https://www.meshlab.net/), an open-source software, is a handy tool to convert common formats to a .obj. 

A note about meshes. Support for meshes as collision geometry is limited as documented [here](https://drake.mit.edu/doxygen_cxx/group__geometry__file__formats.html). In certain cases--point contact, compliant hydroelastic contact, checking collision candidates, signed distance queries--the mesh will be represented by its [convex hull](https://en.wikipedia.org/wiki/Convex_hull) instead. You can convert meshes into sdf's using [manipulation/create_sdf_from_mesh.py](https://github.com/RussTedrake/manipulation/blob/master/manipulation/create_sdf_from_mesh.py), which additionally supports convex decomposition of a nonconvex mesh and mesh simplification.


### Written Question (4 points)
Compare and contrast two SDF files of the same YCB sugar box: [File 1](https://github.com/RobotLocomotion/drake/blob/8ed3a0264c521ffc1ea8861f5701b81dfb696af8/manipulation/models/ycb/sdf/004_sugar_box.sdf) and [File 2](https://github.com/RussTedrake/manipulation/blob/24f122591f72cc78931ea3f461d0c34845dc9aca/manipulation/models/hydro/004_sugar_box.sdf). File 2 uses hydroelastic contact, and file 1 uses the default point contact model.

[Hydroelastic contact](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html) in Drake can be used to create compliant collision geometries. This is a great tool for improving the contact dynamics between simulated objects, compared to traditional rigid contact geometries.

Answer the following questions in your written submission.

**Part (a):**  How is the collision geometry and or contact models different in File 1 than File 2? What benefit do we have from having different collision geometries in both of these versions of the sugar box? (Hint: Which file has hydroelastic contact? You can also look through Chapter 5 specfically [5.2.2-5.2.4](https://manipulation.csail.mit.edu/clutter.html#section2) or read through [this blog post](https://medium.com/toyotaresearch/rethinking-contact-simulation-for-robot-manipulation-434a56b5ec88))

**Part (b):**  In both files, we use the sugar box obj mesh as the visual geometry. For File 2, we could have used the obj mesh as the collision geometry too. why might we prefer to specify the collision geometry using primitives, rather than using the obj mesh?

### Loading and Viewing Models

Drake has some pre-existing models which can be found in the [drake_models repo](https://github.com/RobotLocomotion/models) and in the [manipulation models](https://github.com/RussTedrake/manipulation/tree/master/manipulation/models). You might find some of these useful for your final project!

Typically to add a model to a MultibodyPlant, you will access its [Parser](https://drake.mit.edu/pydrake/pydrake.multibody.parsing.html#pydrake.multibody.parsing.Parser) object ([C++ documentation](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_parser.html)). 

Drake provides a `ModelVisualizer` class to visualize models interactively. This will help as we start to produce our own robot description files, or port description files over from another simulator. In the next few cells, we will go over different ways to add simulated objects or models to your simulation. 

First let's start by adding an existing Drake model--a KUKA iiwa arm you've already seen in class. Under the `Scene > drake` dropdown in MeshCat, you can enable viewing 'illustration', 'inertia', and 'proximity' to see the visual model, inertia matrix, and collision model, respectively. You can also adjust the poses of the links and values of the joints in the sliders. Get familiar playing around with it. 

In [4]:
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(
    url="package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf"
)
visualizer.Run()

INFO:drake:Click 'Stop Running' or press Esc to quit


If there are no existing Drake models that work for you, you can also create your own models or find them on the internet. We define a table top in a string below and then save it to a file. Try editing parts of it to see how the object changes.

In [ ]:
# Create a table top SDFormat model.
table_top_sdf_path = "table_top.sdf"
table_top_sdf = """<?xml version="1.0"?>
<sdf version="1.7">

  <model name="table_top">
  <link name="table_top_link">
    <inertial>
      <mass>18.70</mass>
      <pose>0.0 0.0 0.445 0 0 0</pose>
      <inertia>
          <ixx>0.619</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.374</iyy>
          <iyz>0</iyz>
          <izz>0.99</izz>
      </inertia>
    </inertial>
    <visual name="bottom">
      <pose>0.0 0.0 0.445 0 0 0</pose>
      <geometry>
        <box>
          <size>0.49 0.63 0.015</size>
        </box>
      </geometry>
      <material>
        <diffuse>0.9 0.9 0.9 1.0</diffuse>
      </material>
    </visual>
    <collision name="bottom">
      <pose>0.0 0.0 0.445 0 0 0</pose>
      <geometry>
        <box>
          <size>0.49 0.63 0.015</size>
        </box>
      </geometry>
      <drake:proximity_properties>
        <drake:compliant_hydroelastic/>
        <drake:hydroelastic_modulus>1.0e6</drake:hydroelastic_modulus>
      </drake:proximity_properties>
    </collision>
  </link>
  <frame name="table_top_center">
    <pose relative_to="table_top_link">0 0 0.47 0 0 0</pose>
  </frame>
</model>

</sdf>

"""

with open(table_top_sdf_path, "w") as f:
    f.write(table_top_sdf)

In [ ]:
# You can parse a string directly OR
# You can load a saved SDF file using AddModels
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(table_top_sdf_path)
# visualizer.parser().AddModelsFromString(table_top_sdf_path)
visualizer.Run()

This will create a file in your current directory. It may be a good idea to create an assets folder and put all of your model files in there. If you are collaborating across local machines, one can imagine that using paths to files might get a little unwieldy. In this case, it is a good idea to make your project into a custom xml package--allowing for better referencing of your model files! 

In the cell bellow, we define and save a `package.xml` in the current directory that identifies it as a package with the name `dummy_project`. As long as you register this package with the [Parser's package map](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_package_map.html), you can then call any files within that package using a url the same way we call existing drake models.

In [13]:
pkg_xml = """<?xml version="1.0"?>
<package format="2">
  <name>pizzabot</name>
  <version>0.0.0</version>
  <description>
    pizza pizza
  </description>
  <maintainer email="mattg156@mit.edu">mattg</maintainer>
  <author>mmaisonn</author>
  <license>N/A</license>
</package>
"""

with open("package.xml", "w") as f:
    f.write(pkg_xml)

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().package_map().PopulateFromFolder(os.getcwd())
visualizer.parser().AddModels(url="package://pizzabot/table_top.sdf")
visualizer.Run()

RuntimeError: error: URI 'package://dummy_project/table_top.sdf' resolved to '/Users/mattgardner/source/manipulation/project/notebooks/table_top.sdf' which does not exist.

For simpler objects and projects, sometimes it can be quicker to create your own model from scratch in a string or a new file as above. However, you can also find some 3D models online. Model files can be in any of the formats supported by the [Parser](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_parser.html), though SDF is recommended. Places you can find these models:

- [Drake examples](https://github.com/RobotLocomotion/drake/tree/master/examples)

- Object databases curated for robotics such as YCB. Some are already in drake_models [here](https://github.com/RobotLocomotion/models/tree/master/ycb)!

- [Rubik's Cube](https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/02-Lets-get-you-a-robot-8f86172b-b597-4ceb-9bad-92d11ac7a6cc/notebook/rubiks_cube-35164353b13d4a47910ca14e588c74d6?)

- It is also worth checking [final projects](https://manipulation.csail.mit.edu/misc.html#projects) that had objects you'd like to use, see if their code is open source or ask!

- [TurboSquid](https://www.turbosquid.com/3d-model/free): will often need converting to an .obj file. The .obj file can then be directly turned into a SDF file by using the below terminal command, documented [here](https://drake.mit.edu/pydrake/pydrake.multibody.mesh_to_model.html). On Deepnote, you can open a terminal on the left bar and run the command after you have uploaded your .obj file. 
```sh
python3 -m pydrake.multibody.mesh_to_model --scale=1.0 path/to/mesh.obj
```

## Creating a Simulation

So far we've gone over how to create and examine individual objects. A useful scene will usually have multiple bodies within it. You can put each one in its own model file and load them in one by one to the plant as in the function below. 

We weld the table top to the world frame and set the poses for each free body so they fall onto the table when the simulation starts. Note that we are recording the scene, so you can replay the scenario using the control panel!

In [ ]:
def create_scene(sim_time_step):
    # Clean up the Meshcat instance.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=sim_time_step)
    parser = Parser(plant)
    ConfigureParser(parser)

    # Load the table top we created.
    parser.AddModels(table_top_sdf_path)

    # Load a cracker box from Drake.
    parser.AddModels(url="package://drake_models/ycb/003_cracker_box.sdf")

    # Load a sugar box
    parser.AddModels(url="package://manipulation/hydro/004_sugar_box.sdf")

    # Weld the table to the world so that it's fixed during the simulation.
    table_frame = plant.GetFrameByName("table_top_center")
    plant.WeldFrames(plant.world_frame(), table_frame)

    # Finalize the plant after loading the scene.
    plant.Finalize()

    # We use the default context to calculate the transformation of the table
    # in world frame but this is NOT the context the Diagram consumes.
    plant_context = plant.CreateDefaultContext()

    # Set the initial pose for the free body i,e, the cracker box
    cracker_box = plant.GetBodyByName("base_link_cracker")
    X_WorldTable = table_frame.CalcPoseInWorld(plant_context)
    X_TableCracker = RigidTransform(
        RollPitchYaw(np.asarray([45, 30, 0]) * np.pi / 180), p=[0, 0, 0.8]
    )
    X_WorldCracker = X_WorldTable.multiply(X_TableCracker)
    plant.SetDefaultFreeBodyPose(cracker_box, X_WorldCracker)

    sugar_box = plant.GetBodyByName("base_link_sugar")
    X_TableSugar = RigidTransform(p=[0, -0.25, 0.8])
    X_WorldSugar = X_WorldTable.multiply(X_TableSugar)
    plant.SetDefaultFreeBodyPose(sugar_box, X_WorldSugar)

    # Add visualization to see the geometries.
    AddDefaultVisualization(builder=builder, meshcat=meshcat)

    diagram = builder.Build()
    return diagram


def run_simulation(sim_time_step):
    diagram = create_scene(sim_time_step)
    simulator = Simulator(diagram)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0)
    meshcat.PublishRecording()


run_simulation(0.001)

Alternatively, you can define a [Model Directives](https://github.com/RobotLocomotion/drake/blob/master/multibody/parsing/README_model_directives.md) file. This is a YAML file that defines all your objects in the simulation and weld frames as needed to describe relative poses of your objects. In the code cell below, we define a model directive that does the same as the cell above and loads it into the plant using `LoadModelDirectivesFromString` and `ProcessModelDirectives`[(docs)](https://drake.mit.edu/doxygen_cxx/process__model__directives_8h.html). While not necessary here, it is a common final project setup to use scenarios and HardwareStation.

In [ ]:
# NOTE: I use a file here to avail you of the fact that you can do this
# It is preferable to use the dummy project package we created earlier
# to reference the table_top because using explicit files will need absolute paths.
model_directive = f"""
    directives:
    - add_model:
        name: table_top
        file: file://{os.getcwd()}/table_top.sdf
    - add_weld:
        parent: world
        child: table_top::table_top_center
    - add_model:
        name: cracker_box
        file: package://drake_models/ycb/003_cracker_box.sdf
        default_free_body_pose:
            base_link_cracker:
                translation: [0,0,0.8]
                rotation: !Rpy {{ deg: [42, 33, 18] }}    
    - add_model:
        name: sugar_box
        file: package://manipulation/hydro/004_sugar_box.sdf
        default_free_body_pose:
            base_link_sugar:
                translation: [0,-0.25,0.8]
    """


def create_scene_directives(model_directive, sim_time_step=0.001):
    meshcat.Delete()
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=sim_time_step)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.package_map().Add("dummy_project", os.path.abspath(""))

    directives = LoadModelDirectivesFromString(model_directive)
    models = ProcessModelDirectives(directives, plant, parser)
    plant.Finalize()

    AddDefaultVisualization(builder=builder, meshcat=meshcat)

    diagram = builder.Build()
    return diagram


def run_simulation(sim_time_step):
    diagram = create_scene_directives(model_directive, sim_time_step)
    simulator = Simulator(diagram)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0)
    meshcat.PublishRecording()


run_simulation(0.001)

Using the same YAML you can also create a manipulation scenario to eventually use as a `HardwareStation`. This gives you more tools to work with manipulation setups and have a common project base as most of our questions and examples from the course etc. Using a scenario you can also define the timestep and what kind of contact model you want for your scene (set using the directives file or the [`scenario.plant_config.contact_model`](https://drake.mit.edu/doxygen_cxx/structdrake_1_1multibody_1_1_multibody_plant_config.html)).

In [14]:
# Note that we are using a different sugar box here, which has a hydroelastic contact model defined
model_directive = f"""
    directives:
    - add_model:
        name: table_top
        file: package://dummy_project/table_top.sdf
    - add_weld:
        parent: world
        child: table_top::table_top_center
    - add_model:
        name: cracker_box
        file: package://drake_models/ycb/003_cracker_box.sdf
        default_free_body_pose:
            base_link_cracker:
                translation: [0,0,0.8]
                rotation: !Rpy {{ deg: [42, 33, 18] }}    
    - add_model:
        name: sugar_box
        file: package://manipulation/hydro/004_sugar_box.sdf
        default_free_body_pose:
            base_link_sugar:
                translation: [0,-0.25,0.8]
                

    plant_config:
        time_step: 1e-3
        contact_model: "point"
        discrete_contact_approximation: "sap"
    """


# try changing the directives string above to have hydroelastic contact properties!
def preview_scene_given_directives(model_directive):
    meshcat.Delete()
    scenario = LoadScenario(data=model_directive)
    station = MakeHardwareStation(
        scenario, meshcat, package_xmls=[os.getcwd() + "/package.xml"]
    )

    simulator = Simulator(station)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


preview_scene_given_directives(model_directive)

RuntimeError: error: URI 'package://dummy_project/table_top.sdf' resolved to '/Users/mattgardner/source/manipulation/project/notebooks/table_top.sdf' which does not exist.

## Create your own Simulation Setup (4 points)

Now, you will create a simulation setup of your choosing into simulation. At minimum, this would include a manipulator (robot arm) and an object it might interact with. You have a lot of freedom here. Feel free to lean on examples we used earlier in this notebook! Ideally, this scene starts you off for your intended final project. 

We will require 3 non-robot objects and at least one thing in your simulation set-up cannot be directly taken from the existing models in Drake without modification. There is one point for each object, and one for the robot itself.

In [3]:
# A rough sketch of TODOs to get you started:

# 1. Create or save an object in a file path/string/package url.
#       (Visualizing your model to test it first might be helpful)



# 2. Add your objects to a plant/diagram
# 3. Run a simulator object on this diagram
# try changing the directives string above to have hydroelastic contact properties!


scene = open("objects/environment_setup.yaml")

def preview_scene_given_directives(model_directive):
    meshcat.Delete()
    scenario = LoadScenario(data=model_directive)
    station = MakeHardwareStation(
        scenario, meshcat, package_xmls=[os.getcwd() + "/package.xml"])

    simulator = Simulator(station)
    meshcat.StartRecording()
    simulator.AdvanceTo(2.0 if running_as_notebook else 0.1)
    meshcat.PublishRecording()


preview_scene_given_directives(scene)

FileNotFoundError: [Errno 2] No such file or directory: '../objects/environment_setup.yaml'

In [22]:
!pwd

/Users/mattgardner/source/manipulation/project/notebooks


**Part (c):** In your written submission, briefly describe what the object(s) in your scene are and your process of creating/obtaining them. Take a screenshot of your simulation scene. Your screenshot should include **red contact force arrows** and not green contact force arrows, which indicate that you are using **hydroelastic contact**. 

## How will this notebook be Graded?

If you are enrolled in the class, this notebook will be graded using [Gradescope](www.gradescope.com). You should have gotten the enrollement code on our announcement in Piazza. 

This notebook only consists of written questions, so please submit all responses in your PDF submission to Gradescope. For this exercise, you should have completed the following subparts:
- [2 pts] Correct answers to part (a)
- [2 pts] Correct answers to part (b)
- [4 pts] A screenshot and description for part (c)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=92fbdad2-a4a9-4ef8-b493-c307060b34a1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>